In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reload the dataset
data = pd.read_excel('open_crosion.xlsx')

# Group data by specimen number and count the number of positive class samples in each specimen
specimen_counts = data.groupby('Specimen')['Label'].sum()

specimen_counts

Specimen
1    1210
2     744
Name: Label, dtype: int64

In [4]:
# Identify specimens with only negative classes
negative_class_specimens = specimen_counts[specimen_counts == 0].index

# Create custom k-fold splits based on the described criteria
positive_class_specimens = [1, 2]
folds = []

# Iterate to create folds
for i, pos_specimen in enumerate(positive_class_specimens):
    fold = [pos_specimen]
    # Add two negative class specimens to the fold
    fold.extend(negative_class_specimens[i*2:i*2+2])
    folds.append(fold)

# Leftover specimens
leftovers = list(set(data['Specimen'].unique()) - set([item for sublist in folds for item in sublist]))

folds, leftovers


([[1], [2]], [])

In [16]:
from py4j.java_gateway import launch_gateway

# Start the JVM with increased heap space
port = launch_gateway(javaopts=['-Xmx6g'])  # Set to 4GB

In [17]:
from lolopy.learners import RandomForestRegressor
lolo_rf = RandomForestRegressor()
#lolo_rf.fit(X_train.values, y_train.values)

In [18]:
# Assuming 'Label' is the target variable and the rest are features
X = data.drop('Label', axis=1)
y = data['Label']

# Function to extract data based on the custom fold
def get_data_for_fold(data, fold):
    fold_data = data[data['Specimen'].isin(fold)]
    X_fold = fold_data.drop('Label', axis=1)
    y_fold = fold_data['Label']
    return X_fold, y_fold

# Train and test using custom folds
for i, fold in enumerate(folds):
    # Split the data
    X_train_fold, y_train_fold = get_data_for_fold(data, fold)
    
    # Remaining data is the test set
    test_specimens = list(set(data['Specimen'].unique()) - set(fold))
    X_test_fold, y_test_fold = get_data_for_fold(data, test_specimens)
    
    # Train the model
    lolo_rf.fit(X_train_fold.values, y_train_fold.values)
    
    # Predict on the test set
    predictions, uncertainties = lolo_rf.predict(X_test_fold.values, return_std=True)
    
    # Evaluate the model (you can use your desired metrics here)
    # As a placeholder, I'll print the first few predictions for each fold
    print(f"Fold {i+1} Predictions:", predictions[:5])
    print(f"Fold {i+1} Uncertainties:", uncertainties[:5])
    print('---')


Py4JJavaError: An error occurred while calling o24.train.
: java.lang.OutOfMemoryError: Java heap space


In [19]:
from sklearn.model_selection import train_test_split

# Separate out samples with Label 0 and Label 1
class_0 = data[data['Label'] == 0]
class_1 = data[data['Label'] == 1]

# Randomly sample from the majority class (Class 0) to match the number of samples in Class 1
class_0_undersampled = class_0.sample(n=len(class_1), random_state=42)

# Combine the undersampled Class 0 samples with Class 1 samples
balanced_data = pd.concat([class_0_undersampled, class_1], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)

# Split the balanced dataset into training and testing sets using stratified sampling
X = balanced_data.drop('Label', axis=1)
y = balanced_data['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((3126, 11), (782, 11), (3126,), (782,))

In [21]:
def undersample_data(X, y):
    """Function to undersample majority class to balance the dataset."""
    # Combine X and y for easier sampling
    data = pd.concat([X, y], axis=1)
    
    # Separate out samples with Label 0 and Label 1
    class_0 = data[data['Label'] == 0]
    class_1 = data[data['Label'] == 1]
    
    # Randomly sample from the majority class (Class 0) to match the number of samples in Class 1
    class_0_undersampled = class_0.sample(n=len(class_1), random_state=42)
    
    # Combine the undersampled Class 0 samples with Class 1 samples
    balanced_data = pd.concat([class_0_undersampled, class_1], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Split the combined data back into X and y
    X_balanced = balanced_data.drop('Label', axis=1)
    y_balanced = balanced_data['Label']
    
    return X_balanced, y_balanced

# Train and test using custom folds
for i, fold in enumerate(folds):
    # Split the data
    X_train_fold, y_train_fold = get_data_for_fold(data, fold)
    
    # Perform undersampling on the training data
    X_train_fold, y_train_fold = undersample_data(X_train_fold, y_train_fold)
    
    # Remaining data is the test set
    test_specimens = list(set(data['Specimen'].unique()) - set(fold))
    X_test_fold, y_test_fold = get_data_for_fold(data, test_specimens)
    
    # Train the model
    # Note: I'm commenting out the model training and prediction lines since we don't have lolopy installed.
    # You can uncomment and run them in your local setup.
    lolo_rf.fit(X_train_fold.values, y_train_fold.values)
    
    # Predict on the test set
    predictions, uncertainties = lolo_rf.predict(X_test_fold.values, return_std=True)
    
    # Placeholder print statement to verify the loop
    print(f"Training on Fold {i+1} with {len(X_train_fold)} samples.")
    # Print the first few predictions for each fold
    print(f"Fold {i+1} Predictions:", predictions[:5])
    print(f"Fold {i+1} Uncertainties:", uncertainties[:5])
    print('---')


Py4JJavaError: An error occurred while calling o44.transform.
: java.lang.IllegalArgumentException: java.lang.IllegalArgumentException: Self-suppression not permitted
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(Unknown Source)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(Unknown Source)
	at java.lang.reflect.Constructor.newInstance(Unknown Source)
	at java.util.concurrent.ForkJoinTask.getThrowableException(Unknown Source)
	at java.util.concurrent.ForkJoinTask.reportException(Unknown Source)
	at java.util.concurrent.ForkJoinTask.join(Unknown Source)
	at scala.collection.parallel.ForkJoinTasks$FJTWrappedTask.sync(Tasks.scala:243)
	at scala.collection.parallel.ForkJoinTasks$FJTWrappedTask.sync$(Tasks.scala:243)
	at scala.collection.parallel.AdaptiveWorkStealingForkJoinTasks$AWSFJTWrappedTask.sync(Tasks.scala:304)
	at scala.collection.parallel.ForkJoinTasks.executeAndWaitResu

#Another try

import pandas as pd

# Load the dataset
data = pd.read_excel("open_crosion.xlsx")

# Display the first few rows
data.head()


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Extract features and target
X = data[['Feature 1', 'Feature 2', 'Feature 3', 'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7']]
y = data['Label']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Predict using the trained model
predictions, uncertainties = rf.predict(np.array(X_test), return_std=True)


In [ ]:
import matplotlib.pyplot as plt

# Filter the data by specimen
specimen_1_indices = data['Specimen'][X_test.index] == 1
specimen_2_indices = data['Specimen'][X_test.index] == 2

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(np.arange(sum(specimen_1_indices)), uncertainties[specimen_1_indices], label='Specimen 1', alpha=0.6)
plt.scatter(np.arange(sum(specimen_2_indices)), uncertainties[specimen_2_indices], label='Specimen 2', alpha=0.6)
plt.title('Uncertainty by Specimen')
plt.ylabel('Uncertainty')
plt.xlabel('Sample Index')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()
